In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import quad

np.random.seed(0)

# Exercise 1

In [2]:
c   = 1
mpc = 1

def modular_dist(z, h0, omega_l, omega_k, omega_m, omega_r):

    norm_hubble        = lambda x: np.sqrt(omega_l + omega_k*(1+x)**2 + omega_m*(1+x)**3 + omega_r*(1+x)**4)
    comoving_dist      = quad(norm_hubble, 0, z)
    temp_comoving_dist = np.sinh(np.sqrt(omega_k)*comoving_dist)/np.sqrt(omega_k)
    lumin_dist         = (1+z)*temp_comoving_dist

    return 5*np.log10(lumin_dist) + 25 + 5*np.log10(c/h0/mpc)

def log_likelihood(z_data, modular_dist_data, sigma_data, h0, omega_l, omega_k, omega_m, omega_r):

    likelihood = 0

    for z, mu, sigma in zip(z_data, modular_dist_data, sigma_data):
        likelihood += (modular_dist(z, h0, omega_l, omega_k, omega_m, omega_r) - mu)**2/sigma**2

    return likelihood

## a)